In [ ]:
import numpy as np
import os
import time

from lmfit.models import GaussianModel, Model
from sklearn.linear_model import LinearRegression
from sklearn import mixture, metrics

from shapely.geometry import Polygon

from scipy.stats.distributions import norm
from sklearn.neighbors import KernelDensity
from scipy.stats import gaussian_kde

from matplotlib import pyplot as plt
from matplotlib import cm

from skimage.measure import EllipseModel
from matplotlib.patches import Ellipse

from scipy import ndimage as ndi
from scipy.spatial import distance
from numpy import linalg

import skimage
from skimage import io, transform
from skimage.draw import ellipse
from skimage.color import rgb2gray
from skimage import filters
from skimage.morphology import disk
from skimage import color
from skimage import filters, util
from skimage.morphology import disk, skeletonize, ball

from PIL import Image, ImageDraw, ImageFilter, ImageOps
import copy
import cv2
import os
import sys

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.ndimage import distance_transform_edt as edt
from skimage.draw import ellipse
from skimage.measure import find_contours, approximate_polygon, subdivide_polygon
import logging
import glob
from radio_beam.commonbeam import getMinVolEllipse

from src.utils import grainPreprocess, grainShow, grainMark, grainDraw, grainApprox, grainStats, grainMorphology,grainGenerate
from src.cfg import CfgAnglesNames, CfgBeamsNames, CfgDataset

import plotly.graph_objects as go
from matplotlib.image import NonUniformImage

from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl


## Распределение диаметров

In [ ]:
pixel = 49.7 / 1000
# pixel_1500_1500 = 50 / 983


# images = np.load('data/saved_metadata/images_4x_preprocess.npy')
images = np.load('data/saved_metadata/images_left_(1500,1500,1)_preprocessed.npy')
# images = np.load('artif.npy')

# images = np.load('images_left_(1500,1500,1)_preprocessed.npy')
# images = np.load('data/saved_np/images_10_mixer_preprocess.npy')
names = np.load('data/saved_metadata/metadata_left_(1500,1500,1)_preprocessed.npy')
# names = np.load('data/saved_np/metadata_10_mixer_preprocess.npy')
names = [name.split('\\')[-1] for name in names]

folder = 'data/'

# path = 'data/saved np'
path = 'data/saved_metadata/'
# name = 'artif'
name = 'left_(1500,1500,1)'
# name = '10_mixer_preprocess'
# name = '4x_left_(1500,1500,1)'

types_dict = {'Ultra_Co11': 'средние зерна',
              'Ultra_Co25': 'мелкие зерна',
              'Ultra_Co8': 'средне-мелкие зерна',
              'Ultra_Co6_2': 'крупные зерна',
              'Ultra_Co15': 'средне-мелкие зерна'}

# types_dict = {'class_1': 'Type None'}

for step in range(1, 6):
    # print('step: ',step)
    grainGenerate.diametr_approx_save(path, images,name, names, types_dict, step, pixel=pixel, save=True)

In [ ]:
def diametr_approx_all(folder,name, step, N, M, indices=None,save=False):
    #
    # вывод распределения длин а- и б- полуосей для разных образцов
    #

    xy_scatter = np.load(f'{folder}/' + CfgBeamsNames.values + f'{name}' + f'{step}.npy', allow_pickle=True)
    xy_linear = np.load(f'{folder}/' + CfgBeamsNames.approx + f'{name}' + f'{step}.npy', allow_pickle=True)
    xy_linear_data = np.load(f'{folder}/' + CfgBeamsNames.approx_data + f'{name}' + f'{step}.npy', allow_pickle=True)
    texts = np.load(f'{folder}/' + CfgBeamsNames.legend + f'{name}' + f'{step}.npy', allow_pickle=True)

    if indices is not None:
        xy_scatter = xy_scatter[indices]
        xy_linear = xy_linear[indices]
        xy_linear_data = xy_linear_data[indices]
        texts = texts[indices]

    plt.rcParams['font.size'] = '20'
    gs = gridspec.GridSpec(1, 2)
    # для статьи 12,18
    plt.figure(figsize=(N, M))

    ax1 = plt.subplot(gs[0])
    ax2 = plt.subplot(gs[1])

    # ax1.set_title('распределение a-полуосей связующего вещества', fontsize=20)
    # ax2.set_title('распределение b-полуосей связующего вещества', fontsize=20)

    ax1.set_ylabel('ln(p(x))', fontsize=15)
    ax1.set_xlabel('длина, мкм', fontsize=15)

    ax2.set_ylabel('ln(p(x))', fontsize=15)
    ax2.set_xlabel('длина, мкм', fontsize=15)

    #  ax3.grid()
    markers = ['v', 's', 'D', 'd', 'p', '*']
    for i in range(len(xy_scatter)):
        ax1.plot(xy_linear[i][0][0], xy_linear[i][0][1])
        ax2.plot(xy_linear[i][1][0], xy_linear[i][1][1])

    ax1.legend(texts[:, 0], fontsize=14, loc=1)
    # print(texts[:, 0])
    ax2.legend(texts[:, 1], fontsize=14, loc=1)
    # print(texts[:, 1])

    for i in range(len(xy_scatter)):
        marker = markers[i]
        ax1.scatter(xy_scatter[i][0][0], xy_scatter[i][0][1], marker=marker)
        ax2.scatter(xy_scatter[i][1][0], xy_scatter[i][1][1], marker=marker)

    if save:
        plt.savefig(f'распределение_полуосей_{name}_шаг_{step}')


    plt.show()


In [ ]:
N = 30
M = 15
# для статье N=10,M=5

step = 2
# save=True
save=False

# folder = '../data/saved np'
# folder = 'data/saved_np'
folder = 'data/saved_metadata'
# name = '10_mixer_preprocess'
name = '4x_left_orig_shape_'
# name = '4x_data_border_fix_'
# name = ''

# types_dict = {'Ultra_Co11': 'средние зерна',
#               'Ultra_Co25': 'мелкие зерна',
#               'Ultra_Co8': 'средне-мелкие зерна',
#               'Ultra_Co6_2': 'крупные зерна',
#               'Ultra_Co15': 'средне-мелкие зерна'}




# diametr_approx_all(folder,name, step, N, M, indices=[0, 2, 3])
diametr_approx_all(folder,name, step, N, M, indices=[3,0,2],save=save)

In [ ]:
def diametr_approx_all_bidimentional(folder,name, step, N, M, indices=None,save=False):
    #
    # вывод распределения длин а- и б- полуосей для разных образцов
    #
    pixel = 49.7 / 1000
    xy_scatter = np.load(f'{folder}/' + CfgBeamsNames.values + f'{name}' + f'{step}.npy', allow_pickle=True)
    xy_linear = np.load(f'{folder}/' + CfgBeamsNames.approx + f'{name}' + f'{step}.npy', allow_pickle=True)
    xy_linear_data = np.load(f'{folder}/' + CfgBeamsNames.approx_data + f'{name}' + f'{step}.npy', allow_pickle=True)
    texts = np.load(f'{folder}/' + CfgBeamsNames.legend + f'{name}' + f'{step}.npy', allow_pickle=True)
    all_a_beams = np.load(f'{folder}/' + CfgBeamsNames.original_values_a + f'{name}' + f'{step}.npy', allow_pickle=True)
    all_b_beams = np.load(f'{folder}/' + CfgBeamsNames.original_values_b + f'{name}' + f'{step}.npy', allow_pickle=True)

    if indices is not None:
        xy_scatter = xy_scatter[indices]
        xy_linear = xy_linear[indices]
        xy_linear_data = xy_linear_data[indices]
        texts = texts[indices]
        all_a_beams = all_a_beams[indices]
        all_b_beams = all_b_beams[indices]

    plt.rcParams['font.size'] = '20'
    gs = gridspec.GridSpec(1, 2)
    # для статьи 12,18
    # plt.figure(figsize=(N, M))
    #
    # ax1 = plt.subplot(gs[0])
    # ax2 = plt.subplot(gs[1])

    # ax1.set_title('распределение a-полуосей связующего вещества', fontsize=20)
    # ax2.set_title('распределение b-полуосей связующего вещества', fontsize=20)

    # ax1.set_ylabel('ln(p(x))', fontsize=15)
    # ax1.set_xlabel('длина, мкм', fontsize=15)
    #
    # ax2.set_ylabel('ln(p(x))', fontsize=15)
    # ax2.set_xlabel('длина, мкм', fontsize=15)

    #  ax3.grid()
    markers = ['v', 's', 'D', 'd', 'p', '*']
    # for i in range(len(xy_scatter)):
    #     ax1.plot(xy_linear[i][0][0], xy_linear[i][0][1])
    #     ax2.plot(xy_linear[i][1][0], xy_linear[i][1][1])

    # ax1.legend(texts[:, 0], fontsize=14, loc=1)
    # print(texts[:, 0])
    # ax2.legend(texts[:, 1], fontsize=14, loc=1)
    # print(texts[:, 1])

    # plt.figure(figsize=(8, 8))

    left, width = 8, 2
    bottom, height = 8,2
    spacing = 0.005


    rect_scatter = [left, bottom, width, height]
    rect_histx = [left, bottom + height + spacing, width, 0.4]
    rect_histy = [left + width + spacing, bottom, 0.4, height]

    plt.figure(figsize=(5, 5))

    ax_scatter = plt.axes(rect_scatter)
    ax_scatter.tick_params(direction='in', top=True, right=True)
    ax_histx = plt.axes(rect_histx)
    ax_histx.tick_params(direction='in', labelbottom=False)
    ax_histy = plt.axes(rect_histy)
    ax_histy.tick_params(direction='in', labelleft=False)

    # ax_scatter.scatter(x, y)

    for i in range(len(xy_scatter)):
        marker = markers[i]
        # ax1.scatter(xy_scatter[i][0][0], xy_scatter[i][0][1], marker=marker)
        # ax2.scatter(xy_scatter[i][1][0], xy_scatter[i][1][1], marker=marker)
        ax_scatter.scatter(np.array(all_a_beams[i])*pixel, np.array(all_b_beams[i])*pixel, marker=marker,alpha=0.5)
        ax_histx.hist(np.array(all_a_beams[i])*pixel,bins=100,alpha=0.5,log=True)
        ax_histy.hist(np.array(all_b_beams[i])*pixel,bins=100,alpha=0.5, orientation='horizontal', log=True)

    plt.legend([f'Синий - Ultra_Co6_2 крупные зерна, шаг 2 пикселя, полуосей {len(all_a_beams[0])} ',f'Оранжевый - Ultra_Co11 средние зерна, полуосей {len(all_a_beams[1])}',f'Зеленый - Ultra_Co25 мелкие зерна, полуосей {len(all_a_beams[2])}'], fontsize=14, loc='upper center')

    ax_scatter.set_xlabel('длина b-полуоси, мкм', fontsize=15)
    ax_scatter.set_ylabel('длина a-полуоси, мкм', fontsize=15)
    ax_histx.set_ylabel('ln(N)')
    ax_histy.set_xlabel('ln(N)')

    plt.savefig(f'двойной график полуосей_{name}_шаг_{step}.png')

    plt.show()

    for i in range(len(xy_scatter)):
        fig, ax = plt.subplots()

        a_b_zeros=np.zeros((len(all_a_beams[i]),len(all_b_beams[i])))
        for j in range(len(all_a_beams[i])):
            a_b_zeros[all_a_beams[i][j],all_b_beams[i][j]]+=1

        im, cbar = heatmap(range(max(all_a_beams[i])),range(max(all_b_beams[i])), a_b_zeros, ax=ax,
                           cmap="YlGn", cbarlabel="harvest [t/year]")
        texts = annotate_heatmap(im, valfmt="{x:.1f} t")
        #
        fig.tight_layout()
        plt.show()
    # plt.draw()
    # if save:


In [ ]:
folder = 'data/generated_data//beams'
name = 'left_(1500,1500,1)'
step=2
all_a_beams = np.load(f'{folder}/beams_original_values_a_step_left_(1500,1500,1)2.npy', allow_pickle=True)
all_b_beams = np.load(f'{folder}/beams_original_values_b_step_left_(1500,1500,1)2.npy', allow_pickle=True)

In [ ]:
h_step=1
i=1
pixel = 49.7 / 1000
a_b_zeros=np.zeros((int((max(all_a_beams[i])*pixel))//h_step+1,int((max(all_b_beams[i])*pixel)//h_step+1)))
for j in range(len(all_a_beams[i])):

    key1=all_a_beams[i][j]
    key2=all_b_beams[i][j]

    a_b_zeros[all_a_beams[i][j]//h_step,all_b_beams[i][j]//h_step]+=1

x_sticks=np.arange(a_b_zeros.shape[0]*2,2)
y_sticks=np.arange(a_b_zeros.shape[1]*2,2)

# fig, ax = plt.subplots(figsize=(15,15))
# im = ax.imshow(a_b_zeros)
#
# # ax.set_xticks(np.arange(len(x_sticks)),
# #               x_sticks)
# # ax.set_yticks(np.arange(len(y_sticks)),
# #               labels=y_sticks)
#
# cbar = ax.figure.colorbar(im,
#                           ax = ax,
#                           shrink=0.5 )
# # ax.set_title("Matplotlib Heatmap with imshow", size=20)
# fig.tight_layout()

x=np.array(all_a_beams[i]).reshape((-1))


fig = go.Figure(go.Histogram2d(
    x=all_a_beams[i],
    y=all_b_beams[i]
))
fig.show()

In [ ]:
matrices=[]
step=2
folder = 'data/generated_data/beams'

all_a_beams = np.load(f'{folder}/beams_original_values_a_step_left_(1500,1500,1){step}.npy', allow_pickle=True)
all_b_beams = np.load(f'{folder}/beams_original_values_b_step_left_(1500,1500,1){step}.npy', allow_pickle=True)
name = np.load(f'{folder}/beams_legend_step_left_(1500,1500,1){step}.npy', allow_pickle=True)
print(name)
for  i in range(5):
    x=np.array(all_a_beams[i]).flatten()
    y=np.array(all_b_beams[i]).flatten()

    matrix, *opt = plt.hist2d(x, y, norm=mpl.colors.LogNorm(),bins=50)
    matrices.append(matrix)

In [ ]:
# step3
np.load(f'{folder}/beams_original_values_b_step_left_(1500,1500,1){step}.npy', allow_pickle=True)[0][:100]

In [ ]:
# step1
np.load(f'{folder}/beams_original_values_b_step_left_(1500,1500,1){step}.npy', allow_pickle=True)[0][:100]

In [ ]:

# m1 = np.random.rand(3, 3)
# m2 = np.arange(0, 3*3, 1).reshape((3, 3))

fig=plt.figure(figsize=(25,15))

ax1 = fig.add_subplot(151)
im1 = ax1.imshow(matrices[0], interpolation='None',norm=mpl.colors.LogNorm())
ax1.invert_yaxis()


divider = make_axes_locatable(ax1)
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar =fig.colorbar(im1, cax=cax, orientation='vertical')
ax1.set_ylabel('A-beams length',fontsize=15)
# cbar.set_label('log (number of points)',size=15)

ax2 = fig.add_subplot(152)
im2 = ax2.imshow(matrices[1], interpolation='None',norm=mpl.colors.LogNorm())
ax2.invert_yaxis()

divider = make_axes_locatable(ax2)
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar =fig.colorbar(im2, cax=cax, orientation='vertical')
# cbar.set_label('log (number of points)',size=15)

ax3 = fig.add_subplot(153)
im3 = ax3.imshow(matrices[2], interpolation='None',norm=mpl.colors.LogNorm())
ax3.invert_yaxis()

divider = make_axes_locatable(ax3)
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar =fig.colorbar(im3, cax=cax, orientation='vertical')
# cbar.set_label('log (number of points)',size=15)

ax4 = fig.add_subplot(154)
im4 = ax4.imshow(matrices[3], interpolation='None',norm=mpl.colors.LogNorm())
ax4.invert_yaxis()

divider = make_axes_locatable(ax4)
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar =fig.colorbar(im4, cax=cax, orientation='vertical')
# cbar.set_label('log (number of points)',size=15)

ax5 = fig.add_subplot(155)
im5 = ax5.imshow(matrices[4], interpolation='None',norm=mpl.colors.LogNorm())
ax5.invert_yaxis()

divider = make_axes_locatable(ax5)
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar =fig.colorbar(im5, cax=cax, orientation='vertical')
cbar.set_label('log (number of points)',size=15)

names=['Ultra_Co11 - средние зерна',
       'Ultra_Co15 - средне-мелкие зерна',
       'Ultra_Co25 - мелкие зерна',
       'Ultra_Co6_2 - крупные зерна',
       'Ultra_Co8 - средне-мелкие зерна']

for i,ax in enumerate([ax1,ax2,ax3,ax4,ax5]):
    # ax.set_ylabel('A-beams length',fontsize=15)
    ax.set_xlabel('B-beams length',fontsize=15)
    ax.set_title(names[i],fontsize=15)


# plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
# fig.subplots_adjust(hspace=2)
fig.subplots_adjust(wspace=0.3)
plt.savefig(f'двумерное_распределение_полуосей_шаг_{step}.png')
plt.show()

In [ ]:
N = 30
M = 15
# для статье N=10,M=5

step = 2
# save=True
save=True

# folder = '../data/saved np'
# folder = 'data/saved_np'
folder = 'data/saved_metadata'
# name = '10_mixer_preprocess'
# name = '4x_left_orig_shape_'
name = 'left_(1500,1500,1)'
# name = '4x_data_border_fix_'
# name = ''

# types_dict = {'Ultra_Co11': 'средние зерна',
#               'Ultra_Co25': 'мелкие зерна',
#               'Ultra_Co8': 'средне-мелкие зерна',
#               'Ultra_Co6_2': 'крупные зерна',
#               'Ultra_Co15': 'средне-мелкие зерна'}




# diametr_approx_all(folder,name, step, N, M, indices=[0, 2, 3])
diametr_approx_all_bidimentional(folder,name, step, N, M, indices=[3,0,2],save=save)
# diametr_approx_all_bidimentional(folder,name, step, N, M, indices=[0],save=save)

### Вывод на снимке ненормалных углов

In [ ]:
def draw_edges(image, tol=3, border_eps=5):
    #
    # рисует на изображении линии по точкам контура cnts
    # линии в стиле x^1->x^2,x^2->x^3 и тд
    # на вход подавать обработанное изображение
    #
    new_image = image[:, :, 0]
    im = Image.fromarray(np.uint8(new_image)).convert('RGB')
    draw = ImageDraw.Draw(im)

    cnts = grainMark.get_row_contours(image)
    angles = []

    for j, cnt in enumerate(cnts):
        rules = [image.shape[0] - cnt[:, 0].max() > border_eps, cnt[:, 0].min() > border_eps,
                 image.shape[1] - cnt[:, 1].max() > border_eps, cnt[:, 1].min() > border_eps]
        # проверяем находится ли контур у границы, но это срабатывает очень редко
        if all(rules):
            cnt_aprox = approximate_polygon(cnt, tolerance=tol)
            # основная проверка на неправильные контуры
            if len(cnt_aprox) > 3:
                for i, point in enumerate(cnt_aprox[:-1]):

                    y1, x1 = cnt_aprox[i - 1]
                    y2, x2 = cnt_aprox[i]
                    y3, x3 = cnt_aprox[i + 1]

                    v1 = np.array((x1 - x2, y1 - y2)).reshape(1, 2)
                    v2 = np.array((x3 - x2, y3 - y2)).reshape(1, 2)

                    dot = np.dot(v1[0], v2[0])
                    dist1 = np.linalg.norm(v1[0])
                    dist2 = np.linalg.norm(v2[0])
                    cos = dot / (dist1 * dist2)

                    v = np.concatenate([v1, v2])
                    det = np.linalg.det(v)

                    if abs(cos) < 1:
                        ang = int(np.arccos(cos) * 180 / np.pi)

                        if det < 0:
                            angles.append(ang)
                        else:
                            angles.append(360 - ang)
                    else:
                        if det < 0:
                            angles.append(360)
                            ang = 360
                        else:
                            angles.append(0)
                            ang = 0

                    line_color = (0, 255, 127)

                    if ang <= 30:
                        color = (0, 0, 255)
                        r = 4
                        draw.ellipse((y2 - r, x2 - r, y2 + r, x2 + r), fill=color, width=5)
                        draw.line((y1, x1, y2, x2), fill=line_color, width=4)
                        draw.line((y3, x3, y2, x2), fill=line_color, width=4)

                    elif ang >= 30:
                        color = (255, 0, 0)
                        r = 4
                        draw.ellipse((y2 - r, x2 - r, y2 + r, x2 + r), fill=color, width=5)
                        # draw.line((y1, x1, y2, x2), fill=line_color, width=4)
                        # draw.line((y3, x3, y2, x2), fill=line_color, width=4)


        else:
            continue

    img = np.array(im)

    return img, angles

In [ ]:
# img=io.imread('../data/dataset_4x/Ultra_Co8/Ultra_Co8-001_1.jpeg')
i = 0
img = np.load('data/artif.npy')[1, i]
tol = 3
border_eps = 5
im, angles = draw_edges(img, tol, border_eps)
io.imsave(f'low_angles_artif.png', im)
grainShow.img_show(im, 10, cmap='gray')
